In [37]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast

In [38]:
cluster_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17-vlim_lgt20_desformat_catalog.fit')[1].data)
member_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17-vlim_lgt20_desformat_catalog_members.fit')[1].data)

cluster_table.remove_columns(['P_C','C_LAMBDA','C_LAMBDA_ERR','Q_CEN']) #unknown columns

In [39]:
mcal = fio.FITS('/lsst/troxel/y1a1/mcal-y1a1-combined-riz-unblind-v4-matched.fits',memmap=True,mode='r')[1]
print ("Metacal loaded")
shape_id=mcal['coadd_objects_id'][:]

Metacal loaded


In [40]:
problem_list =['MODEL_MAG', 'MODEL_MAGERR', 'MAG_LAMBDA_ERR', 'PZBINS',
               'PZ', 'RA_CENT', 'DEC_CENT', 'ID_CENT', 'LAMBDA_CHISQ_CENT',
               'ZLAMBDA_CENT', 'P_BCG', 'P_CEN', 'P_FG', 'P_SAT']

In [41]:
band_col_list = []
pz_col_list = []
alt_col_list =[]
for problem in problem_list:
    if cluster_table[problem].shape[1]==4:
        band_col_list.append(problem)
    elif cluster_table[problem].shape[1]==5:
        alt_col_list.append(problem)
    elif cluster_table[problem].shape[1]==21:
        pz_col_list.append(problem)
    else:
        print ("error")

In [42]:
band_list=['g','r','i','z','Y']
pz_index_list=['pz'+str(i+1) for i in range(21)]
alt_index_list=['Alt'+str(i+1) for i in range(5)]

In [43]:
for j in range(4):
    for i,column in enumerate(band_col_list):
        cluster_table[str(('Band',band_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(band_col_list)

In [44]:
for i,column in enumerate(pz_col_list):
    for j in range(21):
        cluster_table[str(('PZ',pz_index_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(pz_col_list)

In [45]:
for i,column in enumerate(alt_col_list):
    for j in range(5):
        cluster_table[str(('Alt',alt_index_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(alt_col_list)

In [46]:
all_col_list=list(cluster_table.columns[0:54])
for i,column in enumerate(all_col_list):
    cluster_table[str(('All','All',column))]=cluster_table[column].data[:]
cluster_table.remove_columns(all_col_list)

In [47]:
for j in range(4):
    for i,column in enumerate(band_col_list[0:2]):
        member_table['('+band_list[j]+','+column+')']=member_table[column].data[:,j]
member_table.remove_columns(band_col_list[0:2]) 

In [48]:
cluster=cluster_table.to_pandas()
member=member_table.to_pandas()
print ("Center& member loaded")
# center_table=None
# member_table=None
print ("Memory cleared")

Center& member loaded
Memory cleared


In [49]:
member_cols = member.columns[0:22]
shape_columns=pd.MultiIndex.from_product([['All'],list(mcal.get_colnames())],names=['Band','Data'])

In [50]:
center_bins=[m+str(n) for m,n in zip(['cen_']*5,np.arange(5)+1)]
center=pd.MultiIndex.from_product([['Center'],center_bins,member_cols])

In [51]:
cluster_band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['MODEL_MAG','MODEL_MAGERR','MAG_LAMBDA_ERR']],
                                     names=['band','data'])

In [52]:
# alt_bins=[m+str(n) for m,n in zip(['alt_']*5,np.arange(5)+1)]
# alt_columns=pd.MultiIndex.from_product([alt_bins,
#                                        alt_col_list.append(shape_names)],
#                                      names=['alt_index','data'])

In [53]:
member_band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['MODEL_MAG','MODEL_MAGERR',]],
                                     names=['Band','Data'])

In [54]:
member_band_columns

MultiIndex([('g',    'MODEL_MAG'),
            ('g', 'MODEL_MAGERR'),
            ('r',    'MODEL_MAG'),
            ('r', 'MODEL_MAGERR'),
            ('i',    'MODEL_MAG'),
            ('i', 'MODEL_MAGERR'),
            ('z',    'MODEL_MAG'),
            ('z', 'MODEL_MAGERR')],
           names=['Band', 'Data'])

In [55]:
shape_index=pd.MultiIndex.from_tuples([('MEM_MATCH_ID','TYPE')],names=['ClusterID','Type'])

In [56]:
shape_df=pd.DataFrame(index=shape_index,columns=member_band_columns)

In [57]:
shape_df.fillna(value=1)

Band                      g                      r                      i  \
Data              MODEL_MAG MODEL_MAGERR MODEL_MAG MODEL_MAGERR MODEL_MAG   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE         1            1         1            1         1   

Band                                   z               
Data              MODEL_MAGERR MODEL_MAG MODEL_MAGERR  
ClusterID    Type                                      
MEM_MATCH_ID TYPE            1         1            1

In [58]:
mcal_df =pd.DataFrame(index=shape_index,columns=shape_columns) 

In [59]:
mcal_df.fillna(value=1)

Band                           All                                         \
Data              coadd_objects_id flags mask_frac psf_e1 psf_e2 psf_size   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE                1     1         1      1      1        1   

Band                                                        ...                \
Data              mcal_psf_e1 mcal_psf_e2 mcal_psf_size e1  ... R22 region ra   
ClusterID    Type                                           ...                 
MEM_MATCH_ID TYPE           1           1             1  1  ...   1      1  1   

Band                                                                         \
Data              dec tilename flags_select flags_select_1p flags_select_1m   
ClusterID    Type                                                             
MEM_MATCH_ID TYPE   1        1            1               1               1   

Band                                               
Data              flags_select_2p flags_select_2m  
ClusterID    Type                                  
MEM_MATCH_ID TYPE               1               1  

[1 rows x 80 columns]

In [60]:
pd.concat([mcal_df,shape_df],axis=1)

Band                           All                                         \
Data              coadd_objects_id flags mask_frac psf_e1 psf_e2 psf_size   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE              NaN   NaN       NaN    NaN    NaN      NaN   

Band                                                          ...  \
Data              mcal_psf_e1 mcal_psf_e2 mcal_psf_size   e1  ...   
ClusterID    Type                                             ...   
MEM_MATCH_ID TYPE         NaN         NaN           NaN  NaN  ...   

Band                                                      g               \
Data              flags_select_2p flags_select_2m MODEL_MAG MODEL_MAGERR   
ClusterID    Type                                                          
MEM_MATCH_ID TYPE             NaN             NaN       NaN          NaN   

Band                      r                      i                      z  \
Data              MODEL_MAG MODEL_MAGERR MODEL_MAG MODEL_MAGERR MODEL_MAG   
ClusterID    Type                                                           
MEM_MATCH_ID TYPE       NaN          NaN       NaN          NaN       NaN   

Band                            
Data              MODEL_MAGERR  
ClusterID    Type               
MEM_MATCH_ID TYPE          NaN  

[1 rows x 88 columns]

In [61]:
member_id=member['ID'].values

In [62]:
# for id in member_id:
#     mask = pd.Series(shape_id)==id

In [63]:
shape_id_series = pd.Series(shape_id)

In [64]:
iloc=pd.Series(np.arange(len(shape_id_series)))

In [65]:
temp=pd.concat([shape_id_series,iloc],axis=1)

In [66]:
def sort2n(x,y):
    """
    Sorts and matches two arrays of object ids where x is unique and y is not (in DES this is coadd_objects_id).
    Slower than sort2().
    """
    
    xsort = np.argsort(x)
    ysort = np.argsort(y)
    i_yx = np.sort(y[np.in1d(y, x, assume_unique=False)])
    i_x = xsort[x[xsort].searchsorted(i_yx)]
    i_y = ysort[y[ysort].searchsorted(i_yx)]
    
    return i_x, i_y

In [67]:
s2n_index=sort2n(member_id,shape_id)

In [68]:
s2n_index[1]

array([       64,        82,       201, ..., 134632602, 134632624,
       134632632])

In [69]:
shape_half=pd.DataFrame(mcal.read(rows=[s2n_index[1]]).byteswap().newbyteorder())

In [70]:
member_half=member.iloc[s2n_index[0]]

In [71]:
member_half_index=pd.MultiIndex.from_tuples(list(zip(['All']*23,member_half.columns[0:23]))+list(member_band_columns),names=['Band','Data']) 

In [72]:
shape_half.columns=shape_columns
member_half.columns=member_half_index
shape_half.set_index(('All','coadd_objects_id'),inplace=True)
member_half.set_index(('All','ID'),inplace=True)

In [73]:
shape_df=pd.concat([member_half,shape_half],axis=1,verify_integrity=True)
shape_df.index.name="ID"

In [74]:
cluster_col_list=[ast.literal_eval(name) for i,name in enumerate(cluster.columns)]
cluster_col=pd.MultiIndex.from_tuples(cluster_col_list, names=['Type','Kind','Data'])

In [79]:
cluster.columns=cluster_col
cluster.set_index(('All', 'All', 'MEM_MATCH_ID'),inplace=True);

In [80]:
cluster.to_pickle('/home/cz136/project/sa/data/cluster.pkl')

In [77]:
shape_df.to_pickle('/home/cz136/project/sa/data/shape.pkl')